In [1]:
pip install pmdarima

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.0 MB/s eta 0:00:00


## Expsmoothing


In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from datetime import timedelta
import pandas as pd
from pmdarima.arima import auto_arima
import numpy as np
from scipy.stats import norm
from statsmodels.tsa.holtwinters import ExponentialSmoothing
def arima(test_week):
    quantiles = np.array([0.01, 0.025] + list(np.arange(0.05, 0.95+0.05, 0.05)) + [0.975, 0.99])
    data_file = '/content/drive/MyDrive/Flu Forecasting/code/smoothedrate.csv'

    prediction_file = '/content/drive/MyDrive/Flu Forecasting/ARIMA/state result/'

    quantiles = np.array([0.01, 0.025] + list(np.arange(0.05, 0.95+0.05, 0.05)) + [0.975, 0.99])


    df = pd.read_csv(data_file)
    df['fips'] = df['fips'].astype(str).str.zfill(2)
    df['Week_end'] = pd.to_datetime(df['Week_end'])
    df = df.drop(['PRCP_mean', 'SNOW_mean', 'TMAXDELTA', 'TMINDELTA','s05_value','scontrol_value'], axis=1)
    #df_train = df[(df['Week_end'] < pd.to_datetime(test_week)) & (df['Week_end'] > pd.to_datetime('2022-08-01'))]
    df_train = df[(df['Week_end'] < pd.to_datetime(test_week))]
    df_train['rate'] = np.log(df_train['rate'] + 1)

    test_states = df_train['fips'].unique()

    quantile_result = pd.DataFrame(columns=['fips', 'Week_end', 'horizon', 'Quantile', 'Value'])

    # Accumulate entries in a list
    results = []

    for fips in test_states:
        df_state_train = df_train[df_train['fips'] == fips].set_index('Week_end').copy()

        #model = auto_arima(df_state_train['rate'], seasonal=False, stepwise=True,
         #                 suppress_warnings=True, error_action="ignore", trace=True, max_order=None, test="adf")

        #model = ARIMA(df_state_train['rate'], order=model.order)

        model = ExponentialSmoothing(df_state_train['rate'], trend='add', seasonal='add', seasonal_periods=52)

        model_fit = model.fit()


        forecast = model_fit.forecast(steps=4)
        forecast_values = np.array(forecast)
        residuals = model_fit.resid
        residual_std = np.std(residuals)

        for horizon in range(0, 4):
            scaled_std = residual_std * np.sqrt(horizon + 1)
            quantile_forecasts = norm.ppf(quantiles, loc=forecast_values[horizon], scale=scaled_std)
            quantile_forecasts = np.exp(quantile_forecasts) - 1

            for q, quantile_forecast in zip(quantiles, quantile_forecasts):
                results.append({
                    'fips': fips,
                    'Week_end': pd.to_datetime(test_week),
                    'horizon': horizon,
                    'Quantile': q,
                    'Value': max(0, quantile_forecast)
                })


    quantile_result = pd.concat([quantile_result, pd.DataFrame(results)], ignore_index=True)
    pop = pd.read_csv('/content/drive/MyDrive/Flu Forecasting/code/population.csv')

    pop['fips'] = pop['fips'].astype(str).str.zfill(2)
    df = quantile_result.merge(pop, on='fips', how='inner')

    df['Value'] = (df['Value'] * df['population'] / 100000).round().astype(int)
    df.drop(columns=['population'], inplace=True)
    df['Value'][df['Value'] < 0] = 0
    quantile_result1 = df.copy()

    quantile_result1.rename(columns={
        'fips': 'location',
        'Week_end': 'reference_date',
        'horizon': 'horizon',
        'Quantile': 'output_type_id',
        'Value': 'value'
    }, inplace=True)
    quantile_result1['value'][quantile_result1['value'] < 0] = 0

    quantile_result1['target'] = 'wk inc flu hosp'

    quantile_result1['horizon'] = quantile_result['horizon'].astype(int)  # Convert 'horizon' to integer if it's not already

    quantile_result1['reference_date'] = pd.to_datetime(quantile_result1['reference_date'])
    quantile_result1['target_end_date'] = quantile_result1['reference_date'] + quantile_result1['horizon'] * pd.to_timedelta(7, unit='D')

    quantile_result1['output_type'] = 'quantile'
    quantile_result1['target_end_date'] = pd.to_datetime(quantile_result1['target_end_date'])
    quantile_result1=quantile_result1[['reference_date', 'target', 'horizon', 'target_end_date', 'location', 'output_type', 'output_type_id', 'value']]
    quantile_result1.to_csv(prediction_file + test_week + '_quantileresultexp.csv',index = False)

In [ ]:
from datetime import datetime, timedelta
start_date = datetime.strptime('2023-10-07', '%Y-%m-%d')
end_date = datetime.strptime('2024-02-24', '%Y-%m-%d')
current_date = start_date
dates_list = []

while current_date <= end_date:
    dates_list.append(current_date.strftime('%Y-%m-%d'))
    current_date += timedelta(days=7)

test_weeks = dates_list

for test_week in test_weeks:
    arima(test_week)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start

In [ ]:
test_weeks

['2023-10-07',
 '2023-10-14',
 '2023-10-21',
 '2023-10-28',
 '2023-11-04',
 '2023-11-11',
 '2023-11-18',
 '2023-11-25',
 '2023-12-02',
 '2023-12-09',
 '2023-12-16',
 '2023-12-23',
 '2023-12-30',
 '2024-01-06',
 '2024-01-13',
 '2024-01-20',
 '2024-01-27',
 '2024-02-03',
 '2024-02-10',
 '2024-02-17',
 '2024-02-24']